# 作家风格识别

<br>
<hr>

# 1.实验介绍

## 1.1实验背景

作家风格是作家在作品中表现出来的独特的审美风貌。  
通过分析作品的写作风格来识别作者这一研究有很多应用，比如可以帮助人们鉴定某些存在争议的文学作品的作者、判断文章是否剽窃他人作品等。  
作者识别其实就是一个文本分类的过程，文本分类就是在给定的分类体系下，根据文本的内容自动地确定文本所关联的类别。
写作风格学就是通过统计的方法来分析作者的写作风格，作者的写作风格是其在语言文字表达活动中的个人言语特征，是人格在语言活动中的某种体现。



## 1.2 实验要求
a）建立深度神经网络模型，对一段文本信息进行检测识别出该文本对应的作者。   
b）绘制深度神经网络模型图、绘制并分析学习曲线。  
c）用准确率等指标对模型进行评估。    



## 1.3 实验环境
可以使用基于 Python 分词库进行文本分词处理，使用 Numpy 库进行相关数值运算，使用 Keras 等框架建立深度学习模型等。



## 1.4 注意事项
+ Python 与 Python Package 的使用方式，可在右侧 `API文档` 中查阅。
+ 当右上角的『Python 3』长时间指示为运行中的时候，造成代码无法执行时，可以重新启动 Kernel 解决（左上角『Kernel』-『Restart Kernel』）。



## 1.5 参考资料
jieba：https://github.com/fxsjy/jieba   
Numpy：https://www.numpy.org/  
Keras: https://keras.io/

# 2.实验内容

## 2.1 介绍数据集

该数据集包含了 8438 个经典中国文学作品片段，对应文件分别以作家姓名的首字母大写命名。  
数据集中的作品片段分别取自 5 位作家的经典作品，分别是：

|序号|中文名|英文名|文本片段个数|
|--|--|--|--|
|1|鲁迅|LX| 1500 条 |
|2|莫言|MY| 2219 条 |
|3|钱钟书|QZS| 1419 条 |
|4|王小波|WXB| 1300 条 |
|5|张爱玲|ZAL| 2000 条 |

+ 其中截取的片段长度在 100~200 个中文字符不等
+ 数据集路径为 "./dataset/"



In [ ]:
# 导入相关包
import os
import time
import random
import jieba as jb
import numpy as np
import jieba.analyse
import tensorflow as tf
import tensorflow.keras as K
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import to_categorical


读取数据集，保存在字典中

In [ ]:
dataset = {}
path = "./dataset/"
files= os.listdir(path)
for file in files:
    if not os.path.isdir(file) and not file[0] == '.': # 跳过隐藏文件和文件夹
        f = open(path+"/"+file, 'r',  encoding='UTF-8'); # 打开文件
        for line in f.readlines():
            dataset[line] = file[:-4]


数据集总共有 8438 个文本片段，现在展示其中的 10 个片段及其作者。

In [ ]:
name_zh = {'LX': '鲁迅', 'MY':'莫言' , 'QZS':'钱钟书' ,'WXB':'王小波' ,'ZAL':'张爱玲'} 
for (k,v) in  list(dataset.items())[:10]:
    print(k,'---',name_zh[v])

## 2.2 数据集预处理  
在做文本挖掘的时候，首先要做的预处理就是分词。  
英文单词天然有空格隔开容易按照空格分词，但是也有时候需要把多个单词做为一个分词，比如一些名词如 "New York" ，需要做为一个词看待。  
而中文由于没有空格，分词就是一个需要专门去解决的问题了。  
这里我们使用 jieba 包进行分词，使用**精确模式**、**全模式**和**搜索引擎模式**进行分词对比。  
更多方法参考：https://github.com/fxsjy/jieba   

In [ ]:
# 精确模式分词
titles = [".".join(jb.cut(t, cut_all=False)) for t,_ in dataset.items()]
print("精确模式分词结果:\n",titles[0])

In [ ]:
# 全模式分词
titles = [".".join(jb.cut(t, cut_all=True)) for t,_ in dataset.items()]
print("全模式分词结果:\n",titles[0])

In [ ]:
# 搜索引擎模式分词
titles = [".".join(jb.cut_for_search(t)) for t,_ in dataset.items()]
print("搜索引擎模式分词结果:\n",titles[0])

### 使用 TF-IDF 算法统计各个作品的关键词频率
TF-IDF（term frequency–inverse document frequency，词频-逆向文件频率）是一种用于信息检索与文本挖掘的常用加权技术。  
* TF-IDF是一种统计方法，用以评估一字词对于一个文件集或一个语料库中的其中一份文件的重要程度。字词的重要性随着它在文件中出现的次数成正比增加，但同时会随着它在语料库中出现的频率成反比下降。  
* TF-IDF的主要思想是：如果某个单词在一篇文章中出现的频率TF高，并且在其他文章中很少出现，则认为此词或者短语具有很好的类别区分能力，适合用来分类。  
这里我们使用 jieba 中的默认语料库来进行关键词抽取，并展示每位作者前 5 个关键词


In [ ]:
import jieba.analyse
# 将片段进行词频统计
str_full = {}
str_full['LX'] = ""
str_full['MY'] = ""
str_full['QZS'] = ""
str_full['WXB'] = ""
str_full['ZAL'] = ""

for (k,v) in dataset.items():
    str_full[v] += k

for (k,v) in str_full.items():
    print(k,":")
    for x, w in jb.analyse.extract_tags(v, topK=5, withWeight=True):
        print('%s %s' % (x, w))

## 2.3 采用 Keras 建立一个简单的深度神经网络模型
通过 Keras 构建深度学习模型的步骤如下：
+ 定义模型——创建一个模型并添加配置层
+ 编译模型——指定损失函数和优化器，并调用模型的 compile() 函数，完成模型编译。
+ 训练模型——通过调用模型的 fit() 函数来训练模型。
+ 模型预测——调用模型的 evaluate()或者 predict() 等函数对新数据进行预测。

### 2.3.1 读取数据集
首先需要读取数据集，记录每个片段的作者并保存。

In [ ]:
# 读取数据和标签
def load_data(path):
    """
    :param path:数据集文件夹路径
    :return:返回读取的片段和对应的标签
    """
    sentences = [] # 片段
    target = [] # 作者
    
    # 定义lebel到数字的映射关系
    labels = {'LX': 0, 'MY': 1, 'QZS': 2, 'WXB': 3, 'ZAL': 4}

    files = os.listdir(path)
    for file in files:
        if not os.path.isdir(file):
            f = open(path + "/" + file, 'r', encoding='UTF-8');  # 打开文件
            for line in f.readlines():
                sentences.append(line)
                target.append(labels[file[:-4]])
                
    target = np.array(target)
    encoder = LabelEncoder()
    encoder.fit(target)
    encoded_target = encoder.transform(target)
    dummy_target = to_categorical(encoded_target)

    return sentences, dummy_target

对读取的片段进行分词，由于分词后的片段任然为中文词语组成的序列，需要创建词汇表，将每个中文词映射为一个数字。这里使用 tf 的 Tokenizer 创建词汇表。



In [ ]:
def padding(text_processed,path, max_sequence_length=80):
    """
    数据处理，如果使用 lstm，则可以接收不同长度的序列。
    :text_processed：不定长的 Token 化文本序列，二维list
    :path：数据集路径
    :max_sequence_length：padding 大小，长句截断短句补 0
    :return 处理后的序列，numpy 格式的二维数组
    """
    res = []
    for text in text_processed:
        if len(text) > max_sequence_length:
            text = text[:max_sequence_length]
        else:
            text = text + [0 for i in range(max_sequence_length-len(text))]
        res.append(text)
    return np.array(res)

In [ ]:
# 查看我们创建词汇表的结果
sentences,target = load_data(path)

#定义是文档的最大长度。如果文本的长度大于最大长度，那么它会被剪切，反之则用0填充
max_sequence_length = 80

# 使用 jieba 机精确模式分词
sentences = [".".join(jb.cut(t, cut_all=False)) for t in sentences]
print(sentences[0])

**Tokenizer** 类允许使用两种方法向量化一个文本语料库： 将每个文本转化为一个整数序列（每个整数都是词典中标记的索引）； 或者将其转化为一个向量，其中每个标记的系数可以是二进制值、词频、TF-IDF权重等。

In [ ]:
# 构建词汇表
vocab_processor = tf.keras.preprocessing.text.Tokenizer(num_words=60000,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ', oov_token='<UNK>')
# 要用以训练的文本列表
vocab_processor.fit_on_texts(sentences)
# 序列的列表，将 sentences 文本序列化
text_processed = vocab_processor.texts_to_sequences(sentences)

将词汇表保存为 json，后续可以直接读取，读取方式为 **tf.keras.preprocessing.text.tokenizer_from_json(json_string)**，可以获得vocab_processor 相同参数的对象

In [ ]:
vocab_json_string = vocab_processor.to_json()
# 将词汇表保存路径
vocab_keras_path = "results/vocab_keras.json"
file = open(vocab_keras_path,"w")
file.write(vocab_json_string)
file.close()

# 将句子 padding 为固定长度，如果使用lstm则不需要 padding 为固定长度
text_processed = padding(text_processed,path)
print(text_processed)

打乱并切分数据集，取 30% 数据作为验证集

In [ ]:
# 验证集比例
val_split = 0.3

# 打乱顺序
text_target = list(zip(text_processed, target))
random.shuffle(text_target)
text_processed[:], target[:] = zip(*text_target)

# 验证集数目
val_counts = int(val_split*len(text_target))

# 切分验证集
val_X = text_processed[-val_counts:]
val_y = target[-val_counts:]
train_X = text_processed[:-val_counts]
train_y = target[:-val_counts]

### 2.3.2 常见的创建模型方式介绍
Keras 的核心数据结构是 model，一种组织网络层的方式。最简单的模型是 [Sequential 顺序模型](https://keras.io/getting-started/sequential-model-guide/)，它由多个网络层线性堆叠。对于更复杂的结构，你应该使用 Keras 函数式 API，它允许构建任意的神经网络图。下面先来看看 Sequential 顺序模型：

In [ ]:
# 方式一: 使用 .add() 方法将各层添加到模型中
# 导入相关包
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

# 选择模型，选择序贯模型（Sequential())
model = Sequential()

# 构建网络层
# 添加全连接层
model.add(Dense(64, input_shape=(max_sequence_length,)))

# 添加激活层，激活函数是 relu
model.add(Activation('relu'))

# 打印模型概况
model.summary()

In [ ]:
# 方式二：网络层实例的列表构建序贯模型
# 导入相关的包
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

# 选择模型，选择序贯模型（Sequential())
# 通过将网络层实例的列表传递给 Sequential 的构造器，来创建一个 Sequential 模型
model = Sequential([
    Dense(64, input_shape=(max_sequence_length,)),
    Activation('relu')
])

# 打印模型概况
model.summary()

In [ ]:
# 方式三：函数式模型
# 导入相关的包
from tensorflow.keras.layers import Input, Dense,Activation
from tensorflow.keras.models import Model

# 输入层，返回一个张量 tensor
inputs = Input(shape=(max_sequence_length,))

# 全连接层，返回一个张量
output_1 = Dense(64)(inputs)

# 激活函数层
predictions= Activation(activation='relu')(output_1)

# 创建一个模型，包含输入层、全连接层和激活层
model = Model(inputs=inputs, outputs=predictions)

# 打印模型概况
model.summary()

### 2.3.3 建立深度学习模型

+ 创建一个简单的深度接神经网络模型用于文本分类

In [ ]:
def dnn_model(train_X, train_y, val_X, val_y, model_save_path='results/demo.h5',
              log_dir="results/logs/"):

    # 选择模型，选择序贯模型（Sequential())
    model = K.Sequential()
    
    # 构建网络层
    # 添加全连接层，输出空间维度64
    model.add(K.layers.Dense(64))

    # 添加激活层，激活函数是 relu
    model.add(K.layers.Activation('relu'))
    model.add(K.layers.Dense(5))
    model.add(K.layers.Activation("softmax"))
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    # 训练模型
    history = model.fit(train_X.astype(np.float64), train_y, batch_size=128, epochs=5,validation_data=(val_X, val_y) )

    # 保存模型
    model.save(model_save_path)

    return history, model

+ 模型训练过程和模型概况

In [ ]:
# 开始时间
start = time.time()

# 数据预处理
data_path = "./dataset/"

# 训练模型，获取训练过程和训练后的模型
history,model = dnn_model(train_X, train_y, val_X, val_y)

# 打印模型概况和模型训练总数长
model.summary()
print("模型训练总时长：",time.time()-start)

+ 模型训练过程图形化


In [ ]:
def plot_training_history(res):
    """
    绘制模型的训练结果
    :param res: 模型的训练结果
    :return:
    """
    # 绘制模型训练过程的损失和平均损失
    # 绘制模型训练过程的损失值曲线，标签是 loss
    plt.plot(res.history['loss'], label='loss')
    
    # 绘制模型训练过程中的平均损失曲线，标签是 val_loss
    plt.plot(res.history['val_loss'], label='val_loss')
    
    # 绘制图例,展示出每个数据对应的图像名称和图例的放置位置
    plt.legend(loc='upper right')
    
    # 展示图片
    plt.show()

    # 绘制模型训练过程中的的准确率和平均准确率
    # 绘制模型训练过程中的准确率曲线，标签是 acc
    plt.plot(res.history['accuracy'], label='accuracy')
    
    # 绘制模型训练过程中的平均准确率曲线，标签是 val_acc
    plt.plot(res.history['val_accuracy'], label='val_accuracy')
    
    # 绘制图例,展示出每个数据对应的图像名称，图例的放置位置为默认值。
    plt.legend()
    
    # 展示图片
    plt.show()

In [ ]:
# 绘制模型训练过程曲线
plot_training_history(history)


+ 加载模型和模型评估


In [ ]:
def load_and_model_prediction(val_X, val_y, model_path = 'results/demo.h5'):
    """
    加载模型和模型评估，打印验证集的 loss 和准确度
    :param validation_generator: 预测数据
    :return: 
    """
    # 加载模型
    model = K.models.load_model(model_path)
    # 获取验证集的 loss 和 accuracy
    loss, accuracy = model.evaluate(val_X, val_y)
    print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy * 100))

In [ ]:
load_and_model_prediction(val_X, val_y)

## 2.4 加载模型并进行预测  

In [ ]:
# 加载demo模型
model = load_model("results/demo.h5")

# 使用阿Q正传的片段测试
sen = "阿Q住在未庄的土谷祠里，给人家打短工度日。\
       虽然常常被村里人开玩笑，但内心他还反过来看不起村里人。\
       他有一个缺点，就是头上有一块癞疮疤。所以只要被人说道有关疮疤的话题，他就发怒。\
       大家觉得他的发怒很有趣，就更加开他的玩笑了。如果觉得对手弱，他就故意找茬吵架。但结果往往是输。"

# 对输入文本进行预处理，使用同一个词汇表进行文本向量化
sen_prosessed = " ".join(jb.cut(sen, cut_all=True))
sen_prosessed = vocab_processor.texts_to_sequences([sen_prosessed])[0]
sen_prosessed = sen_prosessed[:max_sequence_length] if len(sen_prosessed) > max_sequence_length else sen_prosessed + [0 for i in range(max_sequence_length - len(sen_prosessed))]
sen_prosessed = np.array(sen_prosessed).reshape(1,-1)

# 输入模型进行预测
result = model.predict(sen_prosessed)
max_sequence_length
catalogue = list(result[0]).index(max(result[0]))

# 定义int2string的映射关系
labels = {0: '鲁迅', 1: '莫言', 2: '钱钟书', 3: '王小波', 4: '张爱玲'}

print("这是{}的文章".format(labels[catalogue]))

# 3.作业
**作业内容：** 根据一段中文文本（ 100~200 个中文字符），预测这段文本的作者。  


## 3.1 创建并训练模型
深度学习模型训练流程, 包含数据处理、创建模型、训练模型、模型保存、评价模型等。  
如果对训练出来的模型不满意, 你可以通过调整模型的参数等方法重新训练模型, 直至训练出你满意的模型。  
如果你对自己训练出来的模型非常满意, 则可以提交作业!  
可以使用任意一种平台支持的框架进行模型实现。

注意：

1. 你可以在我们准好的接口中实现深度学习模型（若使用可以修改除predict外的函数接口），也可以自己实现深度学习模型或改用其他框架实现,但需要满足predict函数的输入输出符合格式要求！
2. 写好代码后可以在 Py 文件中使用 GPU 进行模型训练。

===========================================实现自己的深度学习模型代码答题区===========================================

双击下方区域开始编写  **数据处理**、**创建模型**、**训练模型**、**保存模型**  和  **评估模型**  等部分的代码，如果使用其他框架训练模型可以自行实现。  

In [ ]:
# 导入相关包
import copy
import pandas as pd
import numpy as np
import tensorflow.keras.backend  as K
from matplotlib import pyplot as plt
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,GRU
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint
from tensorflow.keras import optimizers


def processing_data(data_path, validation_split = 0.3 ):
    """
    数据处理
    :data_path：数据集路径
    :validation_split：划分为验证集的比重
    :return：train_X,train_y,val_X,val_y 
    """
    # --------------- 在这里实现中文文本预处理，包含分词，建立词汇表等步骤 -------------------------------
    train_X, train_y, test_X, test_y = None,None,None,None
    pass

    # --------------------------------------------------------------------------------------------
    
    return train_X, train_y, test_X, test_y

def model(train_X, train_y, val_X, val_y, save_model_path):
    """
    创建、训练和保存深度学习模型
    :param train_X: 训练集特征
    :param train_y: 训练集target
    :param test_X: 测试集特征
    :param test_y: 测试集target
    :param save_model_path: 保存模型的路径和名称
    """
    # --------------------- 实现模型创建、训练和保存等部分的代码 --------------------- 
    pass
    # 保存模型（请写好保存模型的路径及名称）
    
    # --------------------------------------------------------------------------------------------
    
    return 

def evaluate_mode(val_X, val_y, save_model_path):
    """
    加载模型和评估模型
    可以实现，比如: 模型训练过程中的学习曲线，测试集数据的loss值、准确率及混淆矩阵等评价指标！
    主要步骤:
        1.加载模型(请填写你训练好的最佳模型),
        2.对自己训练的模型进行评估

    :param test_X: 测试集特征
    :param test_y: 测试集target
    :param save_model_path: 加载模型的路径和名称,请填写你认为最好的模型
    :return:
    """
    # ----------------------- 实现模型加载和评估等部分的代码 -----------------------
    pass
    
    # ---------------------------------------------------------------------------

def main():
    """
    深度学习模型训练流程,包含数据处理、创建模型、训练模型、模型保存、评价模型等。
    如果对训练出来的模型不满意,你可以通过调整模型的参数等方法重新训练模型,直至训练出你满意的模型。
    如果你对自己训练出来的模型非常满意,则可以提交作业!
    :return:
    """
    data_path = "./dataset"  # 数据集路径
    save_model_path = "results/model.h5"  # 保存模型路径和名称
    validation_split = 0.2 #验证集比重

    # 获取数据、并进行预处理
    train_X, train_y, val_X, val_y = processing_data(data_path, validation_split = validation_split)

    # 创建、训练和保存模型
    model(train_X, train_y, val_X, val_y, save_model_path)

    # 评估模型
    evaluate_mode(val_X, val_y, save_model_path)


if __name__ == '__main__':
    main()

## 3.2 模型预测


注意：
1. 点击左侧栏`提交结果`后点击`生成文件`则只需勾选 `predict()` 函数的cell，即【**模型预测代码答题区域**】的 cell。
2. 请导入必要的包和第三方库 (包括此文件中曾经导入过的)。
3. 请加载你认为训练最佳的模型，即请按要求填写模型路径。
4. `predict()`函数的输入和输出请不要改动。
5. 作业测试时记得填写你的模型路径及名称, 如果采用 [离线任务](https://momodel.cn/docs/#/zh-cn/%E5%9C%A8GPU%E6%88%96CPU%E8%B5%84%E6%BA%90%E4%B8%8A%E8%AE%AD%E7%BB%83%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0%E6%A8%A1%E5%9E%8B) 请将模型保存在 **results** 文件夹下。

=================================  **提交 Notebook 训练模型结果数据处理参考示范**  =================================
<br>


In [ ]:
# 导入相关包
import os
import numpy as np
import jieba as jb
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


# ------------------------------------------------------------------------------
# 本 cell 代码仅为 Notebook 训练模型结果进行平台测试代码示范
# 可以实现个人数据处理的方式，平台测试通过即可提交代码
#  -----------------------------------------------------------------------------

def load_data(path):
    """
    读取数据和标签
    :param path:数据集文件夹路径
    :return:返回读取的片段和对应的标签
    """
    sentences = []  # 片段
    target = []  # 作者

    # 定义lebel到数字的映射关系
    labels = {'LX': 0, 'MY': 1, 'QZS': 2, 'WXB': 3, 'ZAL': 4}

    files = os.listdir(path)
    for file in files:
        if not os.path.isdir(file):
            f = open(path + "/" + file, 'r', encoding='UTF-8');  # 打开文件
            for line in f.readlines():
                sentences.append(line)
                target.append(labels[file[:-4]])

    target = np.array(target)
    encoder = LabelEncoder()
    encoder.fit(target)
    encoded_target = encoder.transform(target)
    dummy_target = to_categorical(encoded_target)

    return sentences, dummy_target


def padding(text_processed, path, max_sequence_length=80):
    """
    数据处理，如果使用 lstm，则可以接收不同长度的序列。
    :text_processed：不定长的 Token 化文本序列，二维list
    :path：数据集路径
    :max_sequence_length：padding 大小，长句截断短句补 0
    :return 处理后的序列，numpy 格式的二维数组
    """
    res = []
    for text in text_processed:
        if len(text) > max_sequence_length:
            text = text[:max_sequence_length]
        else:
            text = text + [0 for i in range(max_sequence_length - len(text))]
        res.append(text)
    return np.array(res)


path = "./dataset/"
sentences, target = load_data(path)
max_sequence_length = 80
# 使用jieba机精确模式分词
sentences = [".".join(jb.cut(t, cut_all=False)) for t in sentences]
vocab_processor = tf.keras.preprocessing.text.Tokenizer(num_words=60000, oov_token='<UNK>')
vocab_processor.fit_on_texts(sentences)
text_processed = vocab_processor.texts_to_sequences(sentences)
# 将句子padding为固定长度，如果使用lstm则不需要padding为固定长度
text_processed = padding(text_processed, path)

===========================================  **模型预测代码答题区域**  =========================================== 

在下方的代码块中编写 **模型预测** 部分的代码，请勿在别的位置作答

In [ ]:
# -------------------------- 请加载您最满意的模型 ---------------------------
# 加载模型(请加载你认为的最佳模型)
# 加载模型,加载请注意 model_path 是相对路径, 与当前文件同级。
# 如果你的模型是在 results 文件夹下的 demo.h5 模型，则 model_path = 'results/demo.h5'
model_path = None

# 加载模型，如果采用 keras 框架训练模型，则 model=load_model(model_path)
model = load_model(model_path)

# ---------------------- 模型预测，注意不要修改函数的输入输出 -------------------------
def predict(text):
    """
    :param text: 中文字符串
    :return: 字符串格式的作者名缩写，比如:'LX'
    """
    # ----------- 实现预测部分的代码，以下样例可代码自行删除，实现自己的处理方式 -----------
    
    labels = {0: 'LX', 1: 'MY', 2: 'QZS', 3: 'WXB', 4: 'ZAL'}
    sen_prosessed = " ".join(jb.cut(text, cut_all=True))
    max_sequence_length = 80
    sen_prosessed = vocab_processor.texts_to_sequences([sen_prosessed])[0]
    if len(sen_prosessed) > max_sequence_length:
        sen_prosessed = sen_prosessed[:max_sequence_length]
    else:
        sen_prosessed += [0 for _ in range(max_sequence_length - len(sen_prosessed))]
    sen_prosessed = np.array(sen_prosessed).reshape(1, -1)
    
    # 加载模型进行预测
    result = model.predict(sen_prosessed)
    prediction = labels[list(result[0]).index(max(result[0]))]
    # --------------------------------------------------------------------------

    return prediction

In [ ]:
sen = "我听到一声尖叫，感觉到蹄爪戳在了一个富有弹性的东西上。定睛一看，不由怒火中烧。原来，趁着我不在，隔壁那个野杂种——沂蒙山猪刁小三，正舒坦地趴在我的绣榻上睡觉。我的身体顿时痒了起来，我的目光顿时凶了起来。"
predict(sen)